In [3]:
import numpy as np
import torch

In [4]:
def one_hot_encoding(array, num_classes=0):
    if num_classes == 0:
        num_classes = len(np.unique(array))
    one_hot_array = np.eye(num_classes)[array.reshape(-1)]
    return one_hot_array

In [7]:
def rps(y,p, num_classes=None):
    if y.ndim == 1:
        assert num_classes is not None
        y = one_hot_encoding(y, num_classes=num_classes)
    else:
        # already one-hot-encoded
        num_classes = y.shape[1]
    return ((np.cumsum(y, axis=-1) - np.cumsum(p, axis=-1))**2).sum(axis=-1)/(num_classes-1)

In [10]:
y = np.array([[0,0,1],[0,0,1]])
p = np.array([[0.3, 0.3, 0.4], [0.3, 0.3, 0.4]])          

rps(y,p)

array([0.225, 0.225])

In [11]:
y = np.array([[0,0,1],[0,0,1],[0,0,1]])
p = np.array([[0.3, 0.3, 0.4], [0.3, 0.3, 0.4], [0.3, 0.3, 0.4]])          

rps(y,p)

array([0.225, 0.225, 0.225])

In [ ]:
torch.nn.functional.one_hot(target, n_classes=3)

In [14]:
def rps_torch(logits, target, reduction = 'none'):
    # assumes tensors of shape:
    # logits = BS x C
    # target = BS, will be one-hot encoded internally using n_classes = logits.shape[1]
    bs, num_classes = logits.shape[0], logits.shape[1]
    target= torch.nn.functional.one_hot(target, num_classes=num_classes)
    # probs = logits.softmaxs(1)
    probs = logits
    rps_loss = ((torch.cumsum(target, dim=-1) - torch.cumsum(probs, dim=-1))**2).sum(dim=-1)/(num_classes-1)
    if reduction == 'none':
        return rps_loss 
    else:
        return rps_loss.mean()

In [15]:
y = np.array([[0,0,1],[0,0,1],[0,0,1]])
p = np.array([[0.3, 0.3, 0.4], [0.2, 0.3, 0.5], [0.1, 0.2, 0.7]])          

rps(y,p)

array([0.225, 0.145, 0.05 ])

In [16]:
p_torch = torch.from_numpy(p)
target_torch = torch.from_numpy(np.array([2,2,2]))

In [17]:
rps_torch(p_torch, target_torch)

tensor([0.2250, 0.1450, 0.0500], dtype=torch.float64)